# TABLE OF CONTENTS

1. [Business Problem](#problem)
2. [Basic Imports](#basic_imports) 
3. [Bank Statements Analysis](#bank_statement)
    1. [Loan, Interest & Future Value](#bank_statement1)
    2. [Cash Flow Measures](#bank_statement2)
4. [Financial Statements Analysis](#ratio)
    1. [Profitability Measures](#ratio1)
    2. [Liquidity Measures](#ratio2)
    3. [Solvency Measures](#ratio3)
    4. [Activity Measures](#ratio4)
5. [Conclusion](#conclusion)


# BUSINESS PROBLEM <a name='problem'><a/>

- The objective here is to do an analysis on a potential customer (company) who requested a loan of IDR 500 million for a tenure of 3 months.
- We are provided by with two primary data sources  of Bank Statement (Debit & Credit Cashflow) & Financial Statement (Balance Sheet & income Statement) 
- The objective of this analysis is to make an informed decision, supported by data, on whether to accept or reject the customer's loan request

# BASIC IMPORTS <a name='basic_imports'><a/>

In [1]:
# importimg modules
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import scorecardpy as sc
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
%load_ext autoreload
%autoreload 2

# format output
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
# importing percentile helper functions
def percentile_(n):
    def percentile(grp):
        return grp.quantile(n / 100)
    percentile.__name__ = f'{n}%'
    return percentile

# importing describer helper functions
def describe_(data):
    listItem = []
    for col in data.columns:
        listItem.append([col, data[col].dtype, data[col].notna().sum(), data[col].isna().sum(),
                        round((data[col].isna().sum()/len(data[col])) * 100,2),
                        data[col].nunique(), list(data[col].drop_duplicates().sample(1).values)]);
    dfDesc = pd.DataFrame(columns=['Data Features', 'Data Type', 'Not-Null Count', 'Null Count', 'Null %', 
                                   'N-Unique', 'Unique Sample'],
                         data=listItem)
    return dfDesc

In [3]:
# importing data
df1 = pd.read_csv('Bank_Statement.csv')
df2 = pd.read_excel('Financial_Statement.xlsx', sheet_name='BS', skiprows=2)
df3 = pd.read_excel('Financial_Statement.xlsx', sheet_name='Income Statement')

# data contingency
bank_df = df1.copy(deep=True)
bs_df = df2.copy(deep=True)
is_df = df3.copy(deep=True)

# BANK STATEMENT ANALYSIS <a name=bank_statement><a/>

In [4]:
# converting excel serial date to datetime
def excel_date_to_datetime(excel_date):
    return dt.datetime(1899, 12, 30) + dt.timedelta(days=excel_date)

bank_df['Transaction Date'] = bank_df['Transaction Date'].apply(excel_date_to_datetime)

In [5]:
# overall view of the first bank statement data
display(bank_df.shape)
display(bank_df.head())
display(bank_df.describe())
describe_(bank_df)

(128, 9)

,Bank Name,Bank Country,Bank Account Number,Channel,Currency,Transaction Date,Debit,Credit,Balance
0,MANDIRI,Indonesia,700009922571,csv,IDR,2023-08-04,500000,0,7184719
1,MANDIRI,Indonesia,700009922571,csv,IDR,2023-08-04,555000,0,6629719
2,MANDIRI,Indonesia,700009922571,csv,IDR,2023-08-06,0,36000000,42629719
3,MANDIRI,Indonesia,700009922571,csv,IDR,2023-08-06,36000000,0,6629719
4,MANDIRI,Indonesia,700009922571,csv,IDR,2023-08-11,5000000,0,1629719


,Bank Account Number,Debit,Credit,Balance
count,128.00,128.00,128.00,128.00
mean,700009922571.00,16024980.51,28992680.22,1528094180.85
std,0.00,54661955.61,160820625.41,764061951.22
min,700009922571.00,0.00,0.00,1074629.00
25%,700009922571.00,6500.00,0.00,1652833682.50
50%,700009922571.00,735000.00,0.00,1687706914.00
75%,700009922571.00,8852976.00,0.00,1910508939.50
max,700009922571.00,500000000.00,1447220250.00,2837894748.00


,Data Features,Data Type,Not-Null Count,Null Count,Null %,N-Unique,Unique Sample
0,Bank Name,object,128,0,0.00,1,[MANDIRI]
1,Bank Country,object,128,0,0.00,1,[Indonesia]
2,Bank Account Number,int64,128,0,0.00,1,[700009922571]
3,Channel,object,128,0,0.00,1,[csv]
4,Currency,object,128,0,0.00,1,[IDR]
5,Transaction Date,datetime64[ns],128,0,0.00,26,[2023-10-13T00:00:00.000000000]
6,Debit,int64,128,0,0.00,62,[1735000]
7,Credit,int64,128,0,0.00,16,[1447220250]
8,Balance,int64,128,0,0.00,124,[1623219]


## A) Loan, Interest & Future Values <a name=bank_statement1><a/>

Future Value is the value of a current asset at a specified date in the future based on an assumed rate of growth or interest. 

In [6]:
# loan request & its assumptions
loan_request = 500_000_000
daily_interest_rate = 0.0001 # assumed highest fintech rate for productive loan (OJK)
days = 90 # 3 month tenure

# future value analysis
future_value = loan_request * (1 + daily_interest_rate) ** days
interest_fee = future_value - loan_request
monthly_interest_service = (loan_request + interest_fee)/3 # monthly payment
final_interest_rate = (future_value/loan_request) - 1
print(f"Total Interest: IDR{interest_fee}")
print(f"Monthly Loan Payment: IDR{monthly_interest_service}")
final_interest_rate

Total Interest: IDR4520083.867979109
Monthly Loan Payment: IDR168173361.28932637


0.009040167735958171

- As it can be seen from the previous informations, the loan of IDR500 million with tenure of 3 months & assumed daily interest rate of 0.01% will result in the final interest fee of around IDR4,5 miilion (the final interest rate being around 0.9% by the end of 3 months)
- Within 3 months, its monthly loan payment should be around IDR168,1 million

## B) Cash Flow Measures <a name=bank_statement2><a/>

In [7]:
# getting the range of transaction dates
print(f"Min Date: {bank_df['Transaction Date'].min()} & Max Date:{bank_df['Transaction Date'].max()}")
print(f"Range of {(bank_df['Transaction Date'].max()- bank_df['Transaction Date'].min()).days} days")

Min Date: 2023-08-04 00:00:00 & Max Date:2023-10-30 00:00:00
Range of 87 days


In [8]:
# getting the average balance and its coefficient of variation
balance_lst = bank_df.iloc[-1]['Balance']
balance_avg = bank_df['Balance'].mean()
balance_std = bank_df['Balance'].std()
CV = balance_std/balance_avg

print(f"Last Balance: IDR{balance_lst}")
print(f"Overall Avg Balance: IDR{balance_avg}")
print(f"Coefficient of Variation (CV): {CV}")

Last Balance: IDR1667550282
Overall Avg Balance: IDR1528094180.8515625
Coefficient of Variation (CV): 0.5000097250521941


In [10]:
# getting the monthly average of debit, credit & balance (in period of 3 months)
avg_debit = round(bank_df['Debit'].sum()/3,2)
avg_credit = round(bank_df['Credit'].sum()/3,2)

print(f"Monthly Avg Debit (Cash-Outflow): IDR{avg_debit}")
print(f"Monthly Avg Credit (Cash-Inflow): IDR{avg_credit}")

Monthly Avg Debit (Cash-Outflow): IDR683732501.67
Monthly Avg Credit (Cash-Inflow): IDR1237021022.67


In [11]:
# getting the avg monthly net operating cash flow
avg_operating_cashflow = (bank_df['Credit'].sum() - bank_df['Debit'].sum())/3
print(f"Monthly Avg Net Operating Cash Flow: IDR{avg_operating_cashflow}")

Monthly Avg Net Operating Cash Flow: IDR553288521.0


In [12]:
# cash coverage & debt burden ratio ratio 
print(f"Cash Coverage Ratio: {avg_operating_cashflow/monthly_interest_service}")
print(f"Debt Burden Ratio: {round(monthly_interest_service/avg_operating_cashflow*100,2)}%")

Cash Coverage Ratio: 3.289989072931232
Debt Burden Ratio: 30.4%


- As it can be seen from the previous informations, the overall average bank balance of the company is IDR1,5 billion with Coefficient of Variation 0,5 (well below 1) which means that not only the company has relatively high average of bank balance (compared to monthly interest payment of IDR168,1 million), but it also has a relatively stabler financial swings & activities. Furthermore, the company's average monthly Net Operating Cashflow is around IDR553 million. With that being said, we can know that the Cash Coverage Ratio is 3.29 & its monthly Debt Burden Ratio 30.4% that needs monitoring. These levels of measurement indicates a moderate initial risk outlook.

# FINANCIAL STATEMENTS ANALYSIS <a name=ratio><a/>


In [13]:
# overall view of the balance sheet data
bs_df

,Current Asset,Unnamed: 1,Unnamed: 2
0,Bank Statement,NaN,1667550282.00
1,Cash,NaN,500000000.00
2,Trade Receivables,NaN,5280000000.00
3,Inventory,NaN,200000000.00
4,Total Current Asset,NaN,7647550282.00
5,Total Fixed Asset,NaN,5000000000.00
6,Total Asset,NaN,12647550282.00
7,NaN,NaN,NaN
8,Current Liabilities,NaN,NaN
9,Supplier Payables,NaN,3520000000.00


In [14]:
# overall view of income statement data
is_df

,Income Statement,Unnamed: 1,Unnamed: 2
0,NaN,NaN,YTD Oct-23
1,Revenue,NaN,NaN
2,NaN,Sales,3137751410
3,NaN,NaN,NaN
4,COGS,NaN,NaN
5,NaN,COGS,2196425987
6,NaN,NaN,NaN
7,Op Cost,NaN,NaN
8,NaN,cost,153245990
9,NaN,NaN,NaN


In [15]:
# making sense of the balance sheet data
# assets
bank_statement = 1667550282.00
cash = 500000000.00
trade_receivables = 5280000000.00
inventory = 200000000.00
total_current_assets = 7647550282.00
total_fixed_assets = 5000000000.00
total_assets = 12647550282.00

# liabilities
supplier_payables = 3520000000.00
bank_loans = 1500000000.00
total_current_liabilities = 5020000000.00
total_liabilities = 5020000000.00

# equity
share_capital = 2000000000.00
add_paid_in_capital = 4500000000.00
income_last_year = 500000000.00
income_this_period = 627550282.00
total_equities = 7627550282.00

# making sense of the income statement data
sales = 3137751410
COGS = 2196425987
operating_expense = 153245990
other_expense = 160529151
net_income = 627550282

In [16]:
# checking overall data validity 
print(net_income == sales - COGS - operating_expense - other_expense)
print(total_assets == total_liabilities + total_equities)

True
True


## A) Profitability Measures <a name=ratio1><a/>

Financial ratios are relationships determined from a firm's financial statements, capturing the business financial information and used for analysis & comparison purposes. In conducting the analysis, we should get some benchmarks from the industry to make sense where the company situation is compared to other companies in the industry. The benchmark that I also use is from USA 2023 https://www.readyratios.com/sec/industry/D/ (the dollar conversion rate at October is assumed around 1 USD = IDR15.871k)

In [17]:
# on profitability
EBIT = sales - COGS - operating_expense
EBIT_margin = EBIT/sales
profit_margin = net_income/sales
ROA = net_income/total_assets
ROE = net_income/total_equities

In [18]:
# on EBIT
print(f"EBIT: IDR{EBIT} or USD{round(EBIT/15871, 2)}")
print(f"EBIT Margin: {round(EBIT/sales *100,2)}%")

EBIT: IDR788079433 or USD49655.31
EBIT Margin: 25.12%


- EBIT or Earnings Before Income & Tax are earnings that's used for checking company's internaal operational efficiency.
- EBIT of IDR788 million, although it is positive, it's seen as relatively low compared to the US manufacturing industry in 2023 yet relatively moderate in Indonesia's. It assumed that this company is a small/niche manufacturing company. With that being said, EBIT Margin of 25% is relatively high since it usually on the range of 5-15% showing that there may be a competitive advantage that this compoany has when it comes to cost management.

In [19]:
# on profit margin
print(f"Profit Margin: {round(profit_margin *100,2)}%")

Profit Margin: 20.0%


- Profit Margin is the proportion of revenue that remains as profit after all expenses. It's a good indicator of how efficient the company is converting revenue to actual profit
- Profit Margin of 20% is considered relatively high. This finding supported the previous statement that this company has a relatively healthy management of cost & its efficiency  

In [20]:
# on ROA
print(f"Return on Assets (ROA): {round(ROA *100,2)}%")

Return on Assets (ROA): 4.96%


- ROA is an indicator of how the company utilizes its assets to generate income, returns on asset investments & the economies of scale it
- ROA of 4.96% is moderately efficient & acceptable for a relatively small manufacturing company. For a bigger-sized company, higher efficiencies might be expected. This level shows that every 1 dollar/rupiah in assets, it generates 0,0496 dollar/rupiah in returns.

In [21]:
# on ROE
print(f"Return on Equities (ROE): {round(ROE *100,2)}%")

Return on Equities (ROE): 8.23%


- ROE is a measure of how the company's stockholders fared. It shows how every dollar/rupiah invested in equity generated x dollar/rupiah in returns. 
- ROE of 8.23% is high compared to industry standards. The fact that this company's ROE exceed ROA reflects how the company's use of financial leverage. This level shows that every 1 dollar/rupiah invested in equity, it generates 0,0823 dollar/rupiah in returns.

## B) Liquidity Measures <a name=ratio1><a/>

In [22]:
# on liquidity (short-term capacity to meet its obligations)
current_ratio = total_current_assets/total_current_liabilities
quick_ratio = (total_current_assets-inventory)/total_current_liabilities
cash_ratio = (cash+bank_statement)/total_current_liabilities
net_working_capital = total_current_assets-total_current_liabilities
net_working_capital_to_assets = net_working_capital/total_assets

In [23]:
# on current ratio
print(f"Current Ratio: {current_ratio}")

Current Ratio: 1.5234163908366534


- Current Ratio is one of the best known and widely used to check the proportion of its current assets compared to its current liabilites. Its also shows how efficient is the company in using its cash & short-term assest. The higher the better.
- Current Ratio of 1.52 times indicates the company has their current assets covering its short-term obligations 1.5x over. It is expected to see Current Ratio of at least 1 in a fairly healthy company in terms of its liquidity. However, compared to industry standards, the company's Current Ratio is relatively low (standard of 2++)

In [24]:
# on quick ratio
print(f"Quick Ratio: {quick_ratio}")

Quick Ratio: 1.4835757533864542


- Quick Ratio is a measure that shows how efficient is the company in using its short-term assets outside its inventory (often the least liquid of all current assets) to pay its short-term liabilities. Same with Current Ratio, The higher the better.
- Quick Ratio of 1.48 times indicates the company has their current assets outside inventory covering its short-term obligations 1.48x over, which is on the average side of industry levels.

In [25]:
# on cash ratio
print(f"Cash Ratio: {cash_ratio}")

Cash Ratio: 0.4317829247011952


- Cash Ratio is a measure that shows how every 1 dollar/rupiah is covered by x dollar/rupiah in cash, indicating how able is the company in paying its current liabilities in cash
- Quick Ratio of 0.43 times indicates the company has 0.43 dollar/rupiah to cover every 1 dollar/rupiah in short-term obligations. This shows how the company has relatively minimal cash relative to its current liabilities (value below 1) implying that it has a risk of facing financial vulnerability with its low cash liquidity

In [26]:
# on net working capital
print(f"Net Working Capital (NWC): IDR{net_working_capital} or USD{round(net_working_capital/15871, 2)}")
print(f"Net Working Capital (NWC) to Total Assets: {net_working_capital/total_assets}")

Net Working Capital (NWC): IDR2627550282.0 or USD165556.69
Net Working Capital (NWC) to Total Assets: 0.20775171661025385


- Net Working Capital (NWC) is a measure that shows the excess of current assets over current liabilities. It indicates the amount of capital available to the company for its day-to-day operations after covering short-term obligations.
- NWC of IDR2,6 billions indicates it has extra current assets around IDR2,6 billions to cover their daily operations & obligations. NWC to Total Assets of 0.21 shows that the company hold around 21% of its total assets in net working capital, which is considered a moderate level of liquidity

## C) Solvency Measures <a name=ratio2><a/>

In [29]:
# on solvency (long-term capability to meet its obligations; financial leverage)
total_debt_ratio = (total_assets-total_equities)/total_assets
debt_equity_ratio = (total_assets-total_equities)/total_equities
equity_multiplier = total_assets/total_equities

In [30]:
# total debt ratio
print(f"Total Debt Ratio: {total_debt_ratio}")

Total Debt Ratio: 0.39691480864436385


- Total Debt Ratio measures the proportion of a company's assets that are financed by debt. It considers all liabilities in relation to the company's total assets. 
- Total Debt Ratio is 0.40 indicates that 40% of company's assets are financed by debt (and 60% is from shareholders' equity). Though this level is still below the industry level & appears relatively balanced, the company should continue to monitor its debt levels and financial performance

In [31]:
# debt to equity ratio
print(f"Debt-to-Equity Ratio: {debt_equity_ratio}")

Debt-to-Equity Ratio: 0.658140531940711


- Debt-to-Equity Ratio measures the proportion of a company's debts in realtion to its equities. Showing for every dollar/rupiah of equity, the company has x dollar/rupiah amount of debt.
- The company's Debt-to-Equity is 0.66, indicating that the company has more equity than debt in its capital structure (value below 1). Lower debt levels generally means lower financial risk, as the company has less debt to service, reducing the likelihood of financial distress.

In [32]:
# on equity multiplier
print(f"Equity Multiplier: {equity_multiplier}")

Equity Multiplier: 1.658140531940711


- Equity Multiplier is an indicator of how the company uses it financial leverage.
- This company's Equity Multiplier is 1.66, indicating that the company's total assets is 1.66x the amount of shareholder's equity. It is still below 2 which means that the company has moderate use of debt.

## D) Activity Measures <a name=ratio2><a/>

In [33]:
# on activity (asset utilization & management measures)
inventory_turnover = COGS/inventory
days_sales_in_inventory = 365/inventory_turnover
receivables_turnover = sales/trade_receivables
days_sales_in_receivables = 365/receivables_turnover
NWC_turnover = sales/net_working_capital

In [34]:
# on inventory turnover
print(f"Inventory Turnover: {inventory_turnover}")
print(f"Days Sales in Inventory: {days_sales_in_inventory}")

Inventory Turnover: 10.982129935
Days Sales in Inventory: 33.235811464654645


- These inventory turnover measures are indicating how fast the company's sell their products
- Inventory Turnover of 10.98 times meaning thath the company sold off the entire inventory around 10-11 times. Day Sales in Inventory of 33.2 days means that on average, inventory sits around 33 days before it is sold. These numbers are quite qood relative to the manufacture industry standards, indicating that the company is relatively efficient in selling & replacing its inventories

In [35]:
# on receivables turnover
print(f"Receivables Turnover: {receivables_turnover}")
print(f"Days Sales in Receivables: {days_sales_in_receivables}")
print(f"Proportion of Trade Receivables in Current Assets: {round(trade_receivables/total_current_assets*100,2)}%")

Receivables Turnover: 0.5942711003787878
Days Sales in Receivables: 614.197795866818
Proportion of Trade Receivables in Current Assets: 69.04%


- These inventory turnover measures are indicating how fast the company's collect on their sales
- Receivables Turnover of 0.59 times meaning that the company collected its outstanding credit accounts 0.59 times (so far, the company haven't recovered it yet). Days Sales in Receivables of 614.2 days means that on average, the company collects its credit sales in 614.2 days. These numbers are very high meaning that its trade receivables of IDR5,2 billion rupiah will take more than 1.5 year to be collected. In other words, it's not that liquid. Moreover, a very high proportion of trade receivables of 69% (relative to all Current Assets) would be an issue since its heavy reliance on collecting receivables to maintain liquidity.

# CONCLUSION <a name='conclusion'><a/>

### Positive Indicators:

Strong profitability (high EBIT and net income margins).
Relatively satisfactory levels of ROA and ROE.
Good liquidity ratios (current and quick ratios above 1) & comparatively reasonable long-term solvency & debt-burden ratios.


### Negative Indicators:

Extremely high days sales in receivables (more than 1.5 years) & high portion of current assets in trade receivables (69%), indicating potential cash flow issues. Cash ratio is comparatively low, though mitigated by a relatively good bank balance & net operating cash flow.

### Recommendation: Accept the loan application with conditions.
- Monitor the collection of trade receivables closely to ensure cash flow is maintained.
- Require a portion of the loan resources to be used specifically for improving the receivables collection process (increasing receivables collection rate).
- Maintain a minimum average monthly bank balance above a set threshold to ensure liquidity (increasing cash ratio).